In [77]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
import datetime
import re

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [30]:

def existedb(url: str):
    """Funcion que verifica si una url existe en la base de datos
    Args:
        url (str): url del articulo a verificar

    Returns:
        Bool: False si la url existe, True si no existe
    """
    
    db = pd.read_csv("../data/raw/semana.csv",encoding='utf8')
    return True if (db["URL"].eq(url)).any() else False
    # with open('../data/raw/articulos.csv', encoding="utf8") as f:
    #     csvreader = csv.reader(f, delimiter=",")
    #     for row in csvreader:
    #         if url in row[5]:
    #              return False
    #     return True


In [2]:
def obtener_tags(driver: sel.webdriver.Edge):
    """Funcion que obtiene los tags del articulo

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str[]: Lista de strings/tags
    """
    tags = []
    try :
        secTags = driver.find_element(By.XPATH,'.//div[contains(@class,"tags-list")]')
    except:
        tags = 'SIN TAGS'
    else:
        units = secTags.find_elements(By.XPATH,'.//span')
        for i in units:
            tags.append(i.text)
                  
    return tags
    

In [79]:
def obtener_contenido(driver: sel.webdriver.Edge):
    """Funcion que itera sobre todos los parrafos del articulo y los extrae.

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: devuelve el contenido del articulo
    """
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    contenido = ''
    try :
        #html = driver.find_element(By.XPATH,'.//article[contains(@class,"paywall")]')
        html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//article[contains(@class,"paywall")]')))
        parrafos = html.find_elements(By.XPATH,'.//p')
    except:
        try:
            html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//p[contains(@id,"textId")]')))
            #html = driver.find_element(By.XPATH,'.//p[contains(@id,"textId")]')
            parrafos = html.find_elements(By.XPATH,'.//p')
        except:
            contenido = 'SIN PARRAFOS'
        else:
            for i in parrafos:
                contenido += i.text
    else:
        for i in parrafos:
            contenido += i.text
       
    return contenido

In [89]:
def obtener_tema(driver: sel.webdriver.Edge):
    """Funcion que itera sobre todos los parrafos del articulo y los extrae.

    Args:
        driver (sel.webdriver.Edge): driver de selenium

    Returns:
        str: devuelve el contenido del articulo
    """
    tema = ''
    try :
        tema = driver.find_element(By.XPATH,'.//h3').text
    except:
        try:
            secc = driver.find_element(By.XPATH,'.//div[contains(@class,"styles__Header-sc-1w6splk-3 jtSkhd hidden-md hidden-sm")]')
            tema = secc.find_element(By.XPATH,'.//h1').text
        except:
            tema = 'SIN TEMA' 
    return tema

In [5]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [97]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.semana.com/buscador/?query={empresa}')
time.sleep(2)

In [98]:
# Extrae la lista de todos los articulos de la pagina
articulos = driver.find_elements(By.XPATH,'.//div[contains(@class,"queryly_item_row")]')

In [99]:
# Itera por cada articulo y extrae la informacion (EN CASO DE QUE NO EXISTA ARCHIVO DE ALMACENAMIENTO ANTERIOR)
titulares = []
for art in articulos:
    url = art.find_element(By.XPATH,'.//a').get_attribute('href')
    fechaP = art.find_element(By.XPATH,'.//div[contains(@style,"margin-bottom:0px;color:#555;font-size:12px;")]').text
    resumen = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_description")]').text
    titulo = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_title")]').text
    txtImage = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_advanced_item_imagecontainer")]').get_attribute('style')
    imagen = 'https://www.semana.com'
    imagen = imagen + txtImage.split("\"")[1]
    titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo':titulo,
                    'Fecha Publicacion':fechaP,
                    'Resumen':resumen,
                    'URL':url,
                    'Imagen':imagen,
                    'Empresa':empresa})
        
    

In [101]:
# busca los autores de cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])

    # agregar contenido al dict de titulares
    tit['Contenido'] = obtener_contenido(driver)

    # agregar tags al dict de titulares
    tit['Tags'] = obtener_tags(driver)
    
    # agregar contenido al dict de titulares
    tit['Tema'] = obtener_tema(driver)
      

    # se podria agregar un if resumen vacio, llamar a resumen. (para las 3 noticias principales)

In [103]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Tema','Resumen','URL','Imagen','Empresa','Contenido','Tags']

In [53]:
# Para agregar titulares a un archivo.
with open('../data/raw/noticias.csv', 'a', newline='') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [105]:
#Para crear un nuevo archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/semana.csv',index=False, encoding='utf8')